![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fdata-viz-of-the-week&branch=main&subPath=stock-prices/stock-prices-ML.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Callysto’s Weekly Data Visualization

## Stock Prices ML

### Recommended Grade levels: 10-12
<br>

### Instructions

Click "Cell" and select "Run All".

This will import the data and run all the code, so you can see this week's data visualization. Scroll back to the top after you’ve run the cells.

![instructions](https://github.com/callysto/data-viz-of-the-week/blob/main/images/instructions.png?raw=true)

**You don't need to do any coding to view the visualizations**.

The plots generated in this notebook are interactive. You can hover over and click on elements to see more information. 

Email contact@callysto.ca if you experience issues.

### About this Notebook

Callysto's Weekly Data Visualization is a learning resource that aims to develop data literacy skills. We provide Grades 5-12 teachers and students with a data visualization, like a graph, to interpret. This companion resource walks learners through how the data visualization is created and interpreted by a data scientist. 

The steps of the data analysis process are listed below and applied to each weekly topic.

1. Question - What are we trying to answer?
2. Gather - Find the data source(s) you will need. 
3. Organize - Arrange the data, so that you can easily explore it. 
4. Explore - Examine the data to look for evidence to answer the question. This includes creating visualizations. 
5. Interpret - Describe what's happening in the data visualization. 
6. Communicate - Explain how the evidence answers the question. 

## Question

How can we utilize machine learning to leverage historical data for predictive analysis using stock-data.

### Goal

We will be using machine-learning to see if we can create models that would be useful in predicting stock prices.

### Background

Understanding financial literacy in the form of stocks at a young age cultivates a foundation for long-term financial stability, promoting individuals to make informed decisions, foster savings habits, and capitalize on the compounding benefits of early investments.

## Gather

Stock prices in this notebook are obtained using the Python library [yfinance](https://pypi.org/project/yfinance/) and stock symbols and names are obtained from the [Nasdaq](https://www.nasdaq.com/market-activity/stocks/screener).

### **Disclaimer**

This notebook is **strictly** for educational and informational purposes and does not constitute financial advice, recommendation, or endorsement. The content presented here is not intended to influence any investment decisions, and readers are strongly advised to seek independent financial advice and conduct their own research before making any investment choices. The authors and contributors of this notebook shall not be held responsible for any financial losses or decisions made based on the information provided.

### Code: 

Run the code cells below to import the libraries we need for this project. Libraries are pre-made code that make it easier to analyze our data.

In [ ]:
import pandas as pd
import plotly.express as px

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

try:
    from xgboost import XGBClassifier
except:
    !pip install xgboost
    from xgboost import XGBClassifier

try:
    import yfinance as yf
except:
    !pip install yfinance
    import yfinance as yf

print("Libaries imported.")

### Data

In this notebook similarly to our non-ML version of stock prices, we'll be primarily obtaining data using the Python library [yfinance](https://pypi.org/project/yfinance/). As mentioned before, yfinance is useful tool in order to obtain financial data on stocks easily. This helps us analyze the data in detail and draw meaningful insights on price trends.

First, we'll be downloading data based off an stock known as an **ETF**. In case you have forgotten what an ETF is from the first notebook, 

The [Canadian Government](https://www.canada.ca/en/financial-consumer-agency/services/savings-investments/investing-basics.html) defines an **Exchange-Traded Fund** **(ETF)** as: 

"an investment fund that holds assets such as stocks, commodities or bonds. Exchange traded funds trade on stock exchanges and have a value that is similar to the total value of the assets they contain. This means that the value of an exchange traded fund can change throughout the day." 

Essentially, an ETF is a *diversified* investment fund traded on stock exchanges. The main benefit of investing in an ETF is that, since the ETF's price is determined by a combination of various stocks, even if a particular stock performs poorly within the ETF, the idea is that diversification across a variety of stocks mitigates the impact on your overall investment.

The data we're using is from the **S&P 500** ETF, named after the Standard & Poor's 500 index, representing 500 major U.S. companies and providing investors a convenient way to track the overall stock market performance.

### Data

In this notebook, we'll be primarily obtaining data using the Python library [yfinance](https://pypi.org/project/yfinance/). As mentioned before, yfinance is useful tool in order to obtain financial data on stocks easily. This helps us analyze the data in detail and draw meaningful insights on price trends.

**Note**: This notebook is a follow-up to the notebook [stock-prices](stock-prices.ipynb). If you have not completed that notebook, finish going through it first before starting this notebook.

Similarly to the stock-prices notebook, we'll be downloading stock-data from the **S&P 500** ETF. It is one of the most widely followed equity indices and is considered a key indicator of the overall health of the U.S. stock market. 

In [ ]:
data = yf.Ticker("^GSPC")
data = data.history(start="1927-01-01", end='2023-12-31')

data

The `data` dataframe represents financial data, where the `Date` serves as the index for this particular dataframe. A dataframe is similar to a spreadsheet, where we have rows and columns that correspond to entries of data. 

In the context of a dataframe, an index is similar to the row numbers in a spreadsheet but, in this case, it's labeled with specific dates. The other columns, such as `Open`, `High`, `Low`, `Close`, `Volume`, `Dividends`, and `Stock Splits`, provide information corresponding to each date.


- **Date:** This shows the date of the financial data.
- **Open:** The price of the stock at the beginning of the trading day.
- **High:** The highest price the stock reached during the trading day.
- **Low:** The lowest price the stock reached during the trading day.
- **Close:** The price of the stock at the end of the trading day.
- **Volume:** This represents the total number of shares traded during the day.

Please note that `Dividends` and `Stock Splits` are included in the table but won't be used in this notebook, so you can ignore them.

These columns will be relevant to creating a **machine-learning** model related to stock-prices.

Machine-learning, or ML for short, is a way to teach computers how to learn and make decisions without being explicitly programmed for every task. 

In relation to stock-prices, when there are clear trends or patterns on when stock prices increase or decrease in value, we can utilize a machine which can compute predictions based on historical data and identified patterns, enabling it to anticipate potential future movements in stock prices. 

# Organize

Similarly to our previous stock-prices notebook, we'll be again performing **data-cleaning**. 

In case you have forgotten what the term means, data cleaning involves the process of identifying and removing errors, inconsistencies, or missing values in a dataset to ensure that the data is accurate. This also involves removing unused information to enhance clarity and focus on relevant data in our dataframe.

Data cleaning also involves the addition of necessary or derived information, such as creating new columns, transforming existing ones, or imputing missing values using appropriate methods.

In our case, we need to create 2 columns which will be used in the **machine-learning** process. One will be called `Tomorrow`, which is created by shifting the `Close` prices column by one day, effectively representing the stock prices for the next trading day. The other is `Target`, a **binary** column indicating whether the stock price is expected to increase ('1') or decrease ('0') compared to the current day. 

In [ ]:
data["Tomorrow"] = data["Close"].shift(-1)
data["Target"] = (data["Tomorrow"] > data["Close"]).astype(int)
data

In financial datasets, various factors can introduce inconsistencies, and one notable factor is **inflation**. Inflation can distort the real value of financial metrics over time, affecting measures like stock prices and economic indicators. As the purchasing power of currency decreases due to inflation, nominal values might increase, giving a misleading impression of growth. 

In our case, the many wars before the 21st century introduces lots of variance in the form of inflation. As a result, we're focusing on stock data starting from **2000** to combat this issue. 

We've also included a video talking about inflation, and how it affects the economy below.

<div style="text-align: center;">
  <iframe width="500" height="300" src="https://www.youtube.com/embed/T8-85cZRI9o" frameborder="0" allowfullscreen></iframe>
</div>

<div style="text-align: center;">
  Video by: <a href="https://www.youtube.com/@crashcourse" target="_blank">Crash Course</a>
</div>

In [ ]:
data = data[data.index >= "2000-01-01"]
data

# Explore and Interpret

Perfect, now we have a dataset ready for machine learning. 

We will be using the predictors, `Open`, `High`, `Low`, `Close`, `Volume`, `Tomorrow`, and `Target` serve as the input features for our machine learning model. In machine learning, predictors, also known as features or independent variables, are the input variables used to make predictions. They are the characteristics or attributes of the data that the model uses to learn patterns and relationships.  

These features represent different aspects of stock market data, such as opening and closing prices, trading volume, and a column named `Tomorrow`, indicating stock prices for the next trading day.

However, it's important to note that including `Tomorrow` in the predictors introduces a form of **data leakage**. `Tomorrow` essentially reveals information about the future that would not be available in real-world scenarios. This can make the model perform unrealistically well during training but fail to generalize to new, unseen data. Similarly, the column `Target` indiciates whether a stock increased or decreased in a day based off of `Tomorrow`, which means we also should not be using this column.

Let's see what happens when we train a model including the `Tomorrow` and `Target` column.

In [ ]:
bad_data = data.copy()
bad_data = bad_data.dropna()

bad_model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=42)

train = bad_data.iloc[:-100]
test = bad_data.iloc[-100:]

predictors = ["Open", "High", "Low", "Close", "Volume", "Tomorrow", "Target"]
bad_model.fit(train[predictors], train["Target"])

In [ ]:
predictions = bad_model.predict(test[predictors])
predictions = pd.Series(predictions, index=test.index)
precision_score(test["Target"], predictions)

#### Bad Model Evaluation

The precision score of *1.0* (100% accuracy) obtained from our model might seem impressive at first glance, however, it's essential to recognize that this result is likely due to the inclusion of the `Tomorrow` and `Target` column in the predictors during model training.

Let's create a realistic model without the columns `Tomorrow` and `Target`.

In [ ]:
model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=42)

train = data.iloc[:-100]
test = data.iloc[-100:]

predictors = ["Open", "High", "Low", "Close", "Volume"]
model.fit(train[predictors], train["Target"])

In [ ]:
predictions = model.predict(test[predictors])
predictions = pd.Series(predictions, index=test.index)
precision_score(test["Target"], predictions)

#### Realistic Model Evaluation

After retraining the model without including the `Tomorrow` column in the predictors, we obtain a more realistic precision score of *0.508* or *50.8%*. 

In the context of stock prediction models, achieving a precision score around 50% is not uncommon. This suggests about half the time, you are guessing correctly on when your stock is going up in price, and vice-versa, you are guessing about half the time when your stock is going down in price. 

Predicting stock price movements is a challenging task due to the inherent complexity and volatility of financial markets. Even well-designed models may struggle to consistently outperform random chance, especially in short-term predictions. 

#### Model Performance Visualization

We can also visualize how our model performed by comparing instances when it was *correct* and *incorrect*. In the plot below, the <span style="color:red">red line</span> represents the model's predictions (1 for an increase in price, 0 otherwise), while the <span style="color:blue">blue line</span> illustrates what actually happened in the historical stock price data.

In [ ]:
combined = pd.concat([test["Target"], predictions], axis=1)
combined.rename(columns={"Target": "Actual", 0: "Predicted"}, inplace=True)
combined

In [ ]:
px.line(combined, x=combined.index, y=['Actual', 'Predicted'], labels={'index': 'Date', 'value': 'Values'},title='Actual vs Predicted').show()

#### Ways to Improve

Now that we know, on average, how well our model performs and have visualized its performance, let's explore ways to improve our model. One common technique is to leverage the concept of a **rolling average** or **rolling mean** for a specific column. In the context of stock prices, calculating the rolling average involves taking the average value of a certain number of consecutive data points, creating a smoother representation of the data over time. 

This can help capture underlying *trends* and reduce *noise* (random or irrelevant fluctuations or irregularities), potentially enhancing the model's ability to identify meaningful patterns. 

In [ ]:
data_copy = data.copy()

predictors = ["Open", "High", "Low", "Close", "Volume"]
target = "Target"

for col in predictors[:4]:
    data_copy[col + ' Rolling Mean'] = data_copy[col].rolling(window=5).mean()

data_copy.dropna(inplace=True)
data_copy.head()

In the upcoming code cells, we'll try to refine our machine learning model by introducing new features based on the rolling mean columns found above alongside finding the best combination of model settings, also known as **hyperparameters**. To quickly sum up what is being done without being overly complicated, the model's performance is *fine-tuned* by exploring various options for the number of *estimators*, *learning rate*, and *maximum depth* through a grid search. The best-performing model is then selected based on precision, a metric that evaluates how well our model correctly identifies positive cases. 

In [ ]:
X = data_copy[predictors + [col + ' Rolling Mean' for col in predictors[:4]]]
y = data_copy[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
pipeline = Pipeline([('imputer', SimpleImputer(strategy="mean")),('classifier', XGBClassifier(random_state=42))])
param_grid = {'classifier__n_estimators': [50, 100, 200],'classifier__learning_rate': [0.01, 0.1, 0.2],'classifier__max_depth': [3, 4, 5]}
grid_search = GridSearchCV(pipeline, param_grid=param_grid, scoring='precision', cv=5)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)
precision = precision_score(y_test, predictions)

print("Best Hyperparameters:", grid_search.best_params_)
print("Precision Score:", precision)

Looking at the output above, we see that our precision score has increased slightly from about *50%* to now *54.8%*. Although this increment may not initially seem significant, even small improvements in precision are noteworthy in the context of stock price prediction models. 

In general, finding a model that performs extremely well is *difficult* when dealing with stock data. If there was a model that already did this, there would be significant financial implications affecting the entirety of the world! Stock markets are inherently complex and influenced by numerous unpredictable factors and what we're trying to encourage in this notebook is not to find the perfect model to get rich, but rather, to build a realistic understanding of the challenges involved in predicting stock prices. 

# Communicate

Below are some writing prompts to help you reflect on the new information that is presented from the data. When we look at the evidence, think about what you perceive about the information. Is this perception based on what the evidence shows? If others were to view it, what perceptions might they have?

- I used to think ____________________but now I know____________________. 
- I wish I knew more about ____________________. 
- This visualization reminds me of ____________________. 
- I really like ____________________.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)